In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np

from firedrake.petsc import PETSc

In [ ]:
import os
current_path = os.getcwd()
print(current_path)

my_io_path = "/content/drive/MyDrive/"
print(my_io_path)

---
---
## Coupling Navier-Stokes with thermal problem, according to Boussinesq buoyancy.

In [ ]:
# Import mesh: set the path correctly
mesh = Mesh('/content/drive/MyDrive/hotplate.msh')
fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend(loc='upper left')

In [ ]:
# Function spaces
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)
W = MixedFunctionSpace([V, Q])
Z = FunctionSpace(mesh, 'P', 1)
# Data and boundary conditions
nu = Constant(0.1)
k = Constant(1.e-4)
beta = Constant(1.e-3)
g = Constant((0,-9.8))
T_ref = Constant(0)
f = Constant((0.,0.))

u_in = Constant((1.0, 0.0))
bcNS_in = DirichletBC(W.sub(0), u_in, 1)
bcNS_plate = DirichletBC(W.sub(0), Constant((0.0, 0.0)), 32)
bcNS_sym = DirichletBC(W.sub(0).sub(1), Constant(0.0), (4, 31))

bcsNS = (bcNS_in, bcNS_plate, bcNS_sym)

T_plate = Constant(20.0)
T_in = Constant(10.0)
bcT_plate = DirichletBC(Z, T_plate, 32)
bcT_in = DirichletBC(Z, T_in, 1)
bcsT = (bcT_plate,  bcT_in)

### Variational problems

In [ ]:
def nonlinear_iteration_NS(u, v, p, q, nu, beta, g, T_ref, f, T, u_old):
    # Implementing fixed-point method for Navier-Stokes equations with thermal source.
    # u,p   :   TrialFunctions
    # v,q   :   TestFunctions
    # f     :   rhs of NS momentum equation
    # T     :   temperature Function
    # u_old :   advecting velocity Function

    a = nu*inner(grad(u), grad(v))*dx + inner(dot(grad(u), u_old),v)*dx - div(v)*p*dx -div(u)*q *dx
    L = -beta*(T-T_ref)*inner(g,v)*dx + inner(f,v)*dx

    return a, L

def nonlinear_iteration_thermal(T, eta, k, f, u):
    # Implementing fixed-point method for Navier-Stokes equations with thermal source.
    # T     :   TrialFunction
    # eta   :   TestFunction
    # f     :   rhs of thermal equation
    # u     :   advecting velocity Function

    a = k * inner(grad(T), grad(eta)) * dx  \
        + inner(grad(T), u) * eta * dx
    L = f * eta * dx

    return a, L

### Initialization and post-processing setup.

In [ ]:
# Initialization
wh = Function(W)
uh, ph = wh.subfunctions
uh.interpolate(u_in)
Th = Function(Z)
Th.interpolate(T_in)

# Plot of initial guess
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')
fig, ax = plt.subplots()
col = tripcolor(Th, axes=ax)
plt.colorbar(col)
plt.title('temperature')

# vtk output for Paraview
basename = 'multiphysics_'
outfileU = File(my_io_path+"output/velocity.pvd")
outfileP = File(my_io_path+"output/pressure.pvd")
outfileT = File(my_io_path+"output/temperature.pvd")
uh.rename("Velocity")   # this name will be used in Paraview
ph.rename("Pressure")   # this name will be used in Paraview
Th.rename("Temperature")   # this name will be used in Paraview
outfileU.write(uh)
outfileP.write(ph)
outfileT.write(Th)

### Definition of the ***linear*** solvers for each nonlinear iteration.

In [ ]:
u, p = TrialFunctions(W)
v, q = TestFunctions(W)
u_old = Function(V)
u_old.assign(uh)

p_old = Function(Q)
p_old.assign(ph)

a_NS, L_NS = nonlinear_iteration_NS(u, v, p, q, nu, beta, g, T_ref, f, Th, u_old)
pb_NS = LinearVariationalProblem(a_NS, L_NS, wh, bcsNS)
solver_NS =  LinearVariationalSolver(pb_NS)#, solver_parameters=param)

T = TrialFunction(Z)
eta = TestFunction(Z)
T_old = Function(Z)
T_old.assign(Th)

a_T, L_T = nonlinear_iteration_thermal(T, eta, k, Constant(0.0), uh)
pb_T = LinearVariationalProblem(a_T, L_T, Th, bcsT)
solver_T =  LinearVariationalSolver(pb_T)#, solver_parameters=param)

### Iterative algorithm for the solution of the nonlinear problem

In [ ]:
maxit = 100
it = 0
tol = 1e-4
err = tol+1    

while it <= maxit and err > tol:

    it += 1

    solver_NS.solve()
    uh, ph = wh.subfunctions
    solver_T.solve()

    err = (errornorm(uh, u_old, 'H1') / norm(u_old, 'H1') +
           errornorm(Th, T_old, 'L2') / norm(T_old, 'L2'))

    print("Iteration = ", it, " Error = ", err)
    uh.rename("Velocity")
    ph.rename("Pressure")
    Th.rename("Temperature")
    outfileU.write(uh)
    outfileP.write(ph)
    outfileT.write(Th)

    u_old.assign(uh)
    p_old.assign(ph)
    T_old.assign(Th)

if it <= maxit:
    print('Nonlinear solver converged in', it, 'iterations.')
else:
    print('Nonlinear solver did NOT converge!\nRelative error =', err, 'after', it, 'iterations.')

In [ ]:
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')
fig, ax = plt.subplots()
col = tripcolor(Th, axes=ax)
plt.colorbar(col)
plt.title('temperature')

### Point 4: stabilization

In [ ]:
# Only the thermal problem needs to be stabilized -> SUPG

def nonlinear_iteration_thermal(T, eta, k, f, u):
    # Implementing fixed-point method for Navier-Stokes equations with thermal source.
    # T     :   TrialFunction
    # eta   :   TestFunction
    # f     :   rhs of thermal equation
    # u     :   advecting velocity Function

     # coefficient delta_K
    ubar = Function(FunctionSpace(mesh, 'DG', 0))
    ubar.project(sqrt(inner(u_old, u_old)))
    h = CellDiameter(mesh)
    delta_K = Function(ubar.function_space())
    delta_K = 0.2 * h / (ubar+1e-15) # piece-w lin Function


    a = k * inner(grad(T), grad(eta)) * dx  + inner(grad(T), u) * eta * dx \
        +(-delta_K*k*div(grad(T)) + inner(grad(T), u))*(inner(grad(eta), u))*dx #+ 0.5*div(u_old)*eta)*dx
    L = f * eta * dx

    return a, L